# Image Viewer for all images in a batch

In [72]:
import sys
from pathlib import Path
from typing import List

import json
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

from topoutils.constants import PROJECT_DIR
from topoutils.sphere_sampling import get_spherical_coordinates
from topoutils.visibility import Vertex, from_json

sys.path.insert(0, '..')

In [73]:
assets = Path(r'e:\android_projects\ImageViewer')
captured_images = assets.joinpath('capturedImages')
cameras_sfm = PROJECT_DIR.joinpath('cameras', 'cameras.sfm')

## Sample polylines
### Green

In [74]:
green_polyline = np.array([
    [2.097919940948E+00, 1.610359922051E-02, 2.190779924393E+00, 255, 255, 255],
    [1.971269965172E+00, 2.922860085964E-01, 2.525049924850E+00, 255, 255, 255],
    [1.83196E+00, 7.04518E-01, 2.65416E+00, 255, 255, 255],
    [1.789610028267E+00, 1.110649943352E+00, 2.692329883575E+00, 255, 255, 255],
    [1.545259952545E+00, 1.343610048294E+00, 2.792229890823E+00, 255, 255, 255],
    [1.419679999352E+00, 2.067719936371E+00, 2.833199977875E+00, 255, 255, 255],
])

### Yellow

In [75]:
yellow_polyline = np.array([
    [8.786039948463E-01, -1.391369998455E-01, 5.187849998474E+00, 255, 255, 255],
    [8.309419751167E-01, 1.031540036201E-01, 5.227940082550E+00, 255, 255, 255],
    [7.717400193214E-01, 1.450600028038E-01, 5.208700180054E+00, 255, 255, 255],
    [5.449720025063E-01, 8.042230010033E-01, 4.987659931183E+00, 255, 255, 255],
    [4.101229906082E-01, 9.882419705391E-01, 4.878980159760E+00, 255, 255, 255],
    [3.861240148544E-01, 1.497460007668E+00, 4.794400215149E+00, 255, 255, 255],
    [2.997510135174E-01, 1.528069972992E+00, 4.036129951477E+00, 255, 255, 255],
    [4.844210147858E-01, 2.060509920120E+00, 4.073040008545E+00, 255, 255, 255]
])

### Blue

In [76]:
blue_polyline = np.array([
    [-6.572080254555E-01, 1.240900039673E+00, 2.931509971619E+00, 255, 255, 255],
    [-6.939949989319E-01, 1.198469996452E+00, 3.038789987564E+00, 255, 255, 255],
    [-6.402289867401E-01, 1.131819963455E+00, 2.981800079346E+00, 255, 255, 255],
    [-5.913980007172E-01, 1.158139944077E+00, 2.861429929733E+00, 255, 255, 255],
    [-6.572080254555E-01, 1.240900039673E+00, 2.931509971619E+00, 255, 255, 255]
])

### Red

In [77]:
red_polyline = np.array([
    [1.359150052071E+00, -4.094719886780E-01, 1.373669981956E+00, 255, 255, 255],
    [1.052219986916E+00, -1.273890025914E-02, 1.117300033569E+00, 255, 255, 255],
    [7.79443E-01, 1.24142E-01, 1.194797E+00, 255, 255, 255],
    [4.97258E-01, 1.89861E-01, 1.46525E+00, 255, 255, 255],
    [5.874119997025E-01, 4.343109950423E-02, 1.556470036507E+00, 255, 255, 255],
    [6.33452E-01, 1.4838E-02, 1.501833E+00, 255, 255, 255],
    [7.467389702797E-01, -8.026939630508E-02, 1.528660058975E+00, 255, 255, 255],
    [9.68015E-01, -1.91973E-01, 1.548352E+00, 255, 255, 255],
    [1.106259942055E+00, -4.076380133629E-01, 1.612190008163E+00, 255, 255, 255],
    [1.359150052071E+00, -4.094719886780E-01, 1.373669981956E+00, 255, 255, 255]
])

In [78]:
# n_range = [2,4,8,16]
n = 16

In [79]:
visibility_folder = PROJECT_DIR.joinpath('visibility', f'n_{n}')
blue_visibility = from_json(visibility_folder.joinpath('BluePolyline.json'))
red_visibility = from_json(visibility_folder.joinpath('RedPolyline.json'))
yellow_visibility = from_json(visibility_folder.joinpath('YellowPolyline.json'))
green_visibility = from_json(visibility_folder.joinpath('GreenPolyline.json'))

In [80]:
polylines = [
    (green_polyline, green_visibility, (0, 1, 0.5)), 
    (yellow_polyline, yellow_visibility, (1, 1, 0)),
    (red_polyline, red_visibility, (1, 0, 0)),
    (blue_polyline, blue_visibility, (0, 170/255, 1)),
]

In [81]:
output_directory = Path('.').resolve().joinpath(f'output_{n}')
output_directory.mkdir(exist_ok=True)

## Functions

In [82]:
def get_image_coordinates(polyline, K, M):
    polyline = np.hstack((polyline[:,:3], np.ones((polyline.shape[0], 1))))
    points = np.matmul(K, np.matmul(M, polyline.T))
    return np.divide(points, points[-1,:]).astype(int)

In [83]:
def calculate_visibility(vertices: List[Vertex], eye: List[float]) -> np.array:
    n = int(np.sqrt(len(vertices[0].visibility_grid)))
    u, v = get_spherical_coordinates(n)
    P = np.array([[v.x, v.y, v.z] for v in vertices])
    delta = eye - P
    R = np.sqrt(np.sum(np.square(delta), axis=1))
    polar_angle = (np.arccos(delta[:, 2]/R) % np.pi).reshape(-1, 1)
    azimuthal_angle = (np.arctan2(delta[:, 1], delta[:, 0]) % (2 * np.pi)).reshape(-1, 1)
    azimuthal_idx = np.argmin(np.abs(u - azimuthal_angle), axis=1)
    polar_idx = np.argmin(np.abs(v - polar_angle), axis=1)
    poly_vis_idx = polar_idx*n+azimuthal_idx
    nn_visibility = [vertex.visibility_grid[vis_idx] for vis_idx, vertex in zip(poly_vis_idx, vertices)]
    return nn_visibility >= R

In [84]:
def clip(coords, image_width, image_height):
    return np.maximum(np.minimum(coords, np.array([[image_width], [image_height], [1]])), 0)

In [85]:
def plot_polyline(image_coords, node_visibility, edges, color, image_width, image_height):
    if not np.any(node_visibility):
        return 
    plt.scatter(
        x=image_coords[:, node_visibility][0], 
        y=image_coords[:, node_visibility][1], 
        facecolors=color,
        edgecolors=color
    )
    for edge in edges:
        v1_visible = node_visibility[edge.vertex1]
        v2_visible = node_visibility[edge.vertex2]
        coords = clip(image_coords[:, [edge.vertex1, edge.vertex2]], image_width, image_height)
        if v1_visible and v2_visible:
            plt.plot(coords[0], coords[1], '-', c=color)
        elif v1_visible or v2_visible:
            plt.plot(coords[0], coords[1], '--', c=color)


In [86]:
def is_inside_image(curr_image_coords, image_width, image_height):
    return np.logical_and(
        np.logical_and(
            curr_image_coords[0] >= 0, 
            curr_image_coords[0] <= image_width,
        ),
        np.logical_and(
            curr_image_coords[1] >= 0, 
            curr_image_coords[1] <= image_height
        )
    )

## Camera information

In [87]:
cameras = json.load(cameras_sfm.open('r'))

In [88]:
cameras.keys()

dict_keys(['version', 'featuresFolders', 'matchesFolders', 'views', 'intrinsics', 'poses'])

In [89]:
intrinsic = cameras['intrinsics'][0]

In [90]:
K = np.array([
    [float(intrinsic["pxFocalLength"]), 0, float(intrinsic["principalPoint"][0]), 0],
    [0, float(intrinsic["pxFocalLength"]), float(intrinsic["principalPoint"][1]), 0],
    [0, 0, 1, 0]
])

In [91]:
views = {view['poseId'] : {
    'imgName': view['path'][view['path'].rfind('/')+1:].upper(),
    'width': int(view['width']),
    'height': int(view['height'])
} for view in cameras['views']}    

In [92]:
for pose_obj in tqdm(cameras['poses']):
    pose = pose_obj['pose']['transform']
    view = views[pose_obj['poseId']]
    img_name = view['imgName']
    im = plt.imread(captured_images.joinpath(img_name))
    fig, ax = plt.subplots(figsize=(16,12))
    implot = ax.imshow(im)
    plt.axis('off')
    
    R = np.array([float(x) for x in pose["rotation"]]).reshape((3,3), order='F')
    C = np.array([[float(x)] for x in pose["center"]])
    T = - np.matmul(R, C)
    M = np.vstack((np.hstack((R, T)), np.array([0, 0, 0, 1])))
    eye = [float(x) for x in pose["center"]]
    
    image_width, image_height = view['width'], view['height']
    
    for curr_polyline, curr_visibility, curr_color in polylines:
        curr_image_coords = get_image_coordinates(curr_polyline, K, M)
        curr_is_visible = np.logical_and(
            calculate_visibility(curr_visibility.vertices, eye),
            is_inside_image(curr_image_coords, image_width, image_height)
        )
        plot_polyline(curr_image_coords, curr_is_visible, curr_visibility.edges, curr_color, image_width, image_height)
    plt.savefig(output_directory.joinpath(img_name), bbox_inches='tight')
    plt.close()

100%|██████████████████████████████████████████████████████████████████████████████████| 77/77 [02:17<00:00,  1.78s/it]
